In [1]:
import jax
import jax.numpy as jnp
from jax import make_jaxpr

from genjax import gen, normal, pjax

In [2]:
@gen
def model(v):
    x = normal(v, 1.0) @ "x"

type(model)

genjax._src.generative_functions.fn.Fn

In [3]:
jax.vmap(model.simulate, in_axes=(0,))((jnp.ones(10),))

FnTrace(
  gen_fn=Fn(
    source_program=Closure(dyn_args=(), fn=<function model at 0x121626480>),
  ),
  args=(
    <jax.Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
  ),
  retval=None,
  subtraces={
    'x': DistributionTrace(
      distribution=genjax.normal(),
      args=(
        <jax.Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
        <jax.Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32, weak_type=True)>,
      ),
      value=<jax.Array float32(10,) ≈-0.45 ±3e-08 [≥-0.45, ≤-0.45] nonzero:10>,
    ),
  },
)

In [4]:
make_jaxpr(jax.vmap(model.simulate, in_axes=(0,)))((jnp.ones(10),))

{ lambda ; a:f32[10]. let
    b:f32[10] = pjax.sample[dist=Normal] a 1.0
    c:f32[10] = broadcast_in_dim[
      broadcast_dimensions=()
      shape=(10,)
      sharding=None
    ] 1.0
  in (a, a, c, b) }

In [5]:
make_jaxpr(pjax.vmap(model.simulate, in_axes=(0,)))((jnp.ones(10),))

{ lambda ; a:f32[10]. let
    b:f32[10] = broadcast_in_dim[
      broadcast_dimensions=()
      shape=(10,)
      sharding=None
    ] 1.0
    c:f32[10] = pjax.sample[dist=Normal] a b
  in (a, a, 1.0, c) }